In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge 
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df1= pd.read_csv('green_tripdata-one-.csv')
df1

In [ ]:
df2= pd.read_csv('green_tripdata_two-.csv')
df2

In [ ]:
con_data= pd.concat([df1,df2])   # concat two dataset
con_data.head(10)

In [ ]:
con_data.shape

In [ ]:
con_data.info()

---
### working on datetime

In [ ]:
con_data['lpep_pickup_datetime'].nunique()

In [ ]:
con_data['lpep_dropoff_datetime'].nunique()

In [ ]:
# convert type to datetime
con_data['lpep_pickup_datetime']= pd.to_datetime(con_data['lpep_pickup_datetime'],format="%Y-%m-%d %H:%M:%S")

In [ ]:
# convert type to datetime
con_data['lpep_dropoff_datetime']= pd.to_datetime(con_data['lpep_dropoff_datetime'],format="%Y-%m-%d %H:%M:%S") 

In [ ]:
con_data.dtypes

In [ ]:
#extract month

con_data["month"] = pd.DatetimeIndex(con_data["lpep_pickup_datetime"]).month
con_data['month_name']=con_data['month'].map({1:"JAN",2:"FEB"})

#extract week day 
con_data["week_day_pickup"]= con_data["lpep_pickup_datetime"].dt.weekday

#extract day 
con_data["day_pickup"]= con_data["lpep_pickup_datetime"].dt.day

#extract hour
con_data["hour_pickup"]= con_data["lpep_pickup_datetime"].dt.hour 

con_data =con_data.sort_values(by = "lpep_pickup_datetime",ascending = False)


In [ ]:
con_data.head()

In [ ]:
#extract month

con_data["month_dropoff"] = pd.DatetimeIndex(con_data["lpep_dropoff_datetime"]).month
con_data['month_name_dropoff']=con_data['month'].map({1:"JAN",2:"FEB"})

#extract week day 
con_data["week_day_dropoff"]= con_data["lpep_dropoff_datetime"].dt.weekday

#extract day 
con_data["day_dropoff"]= con_data["lpep_dropoff_datetime"].dt.day

#extract hour
con_data["hour_dropoff"]= con_data["lpep_dropoff_datetime"].dt.hour 

con_data =con_data.sort_values(by = "lpep_dropoff_datetime",ascending = False)


In [ ]:
con_data.head()

In [ ]:
con_data.sample(4)

In [ ]:
con_data.drop(columns=['month_name', 'month_name_dropoff'], inplace=True)

In [ ]:
con_data.describe()

---
---
### working on (VendorID, store_and_fwd_flag, RatecodeID, payment_type, trip_type) columns to get dummies 

In [ ]:
con_data.shape

In [ ]:
con_data['VendorID'].nunique()

In [ ]:
con_data['VendorID'].dtypes

In [ ]:
# VendorID type conversion to get dummies
con_data['VendorID'] = con_data.VendorID.astype('category')

In [ ]:
con_data['store_and_fwd_flag'].nunique()

In [ ]:
con_data['store_and_fwd_flag'].dtypes

In [ ]:
con_data['RatecodeID'].nunique()

In [ ]:
con_data['RatecodeID'].dtypes

In [ ]:
# RatecodeID type conversion to get dummies
con_data['RatecodeID'] = con_data.RatecodeID.astype('category')

In [ ]:
con_data['payment_type'].nunique()

In [ ]:
con_data['payment_type'].dtypes

In [ ]:
# payment_type type conversion to get dummies
con_data['payment_type'] = con_data.payment_type.astype('category')

In [ ]:
con_data['trip_type'].nunique()

In [ ]:
con_data['trip_type'].dtypes

In [ ]:
# trip_type type conversion to get dummies
con_data['trip_type'] = con_data.trip_type.astype('category')

In [ ]:
con_data = pd.get_dummies(con_data)

In [ ]:
con_data

In [ ]:
con_data.shape

In [ ]:
con_data.columns

In [ ]:
# renames the columns.
con_data.rename(columns={'VendorID_1.0': 'creative_mobile_technologies_LLC',
                         'VendorID_2.0': 'verifone_inc',
                         'store_and_fwd_flag_N': 'not_a_store_and_forward_trip',
                         'store_and_fwd_flag_Y': 'store_and_forward_trip',
                         'RatecodeID_1.0': 'standard_rate',
                         'RatecodeID_2.0': 'JFK',
                         'RatecodeID_3.0': 'newark',
                         'RatecodeID_4.0': 'nassau_or_westchester',
                         'RatecodeID_5.0': 'negotiated_fare',
                         'RatecodeID_99.0': 'group_ride',
                         'payment_type_1.0': 'credit_card',
                         'payment_type_2.0': 'cash',
                         'payment_type_3.0': 'no_charge',
                         'payment_type_4.0': 'dispute',
                         'payment_type_5.0': 'unknown',
                         'trip_type_1.0': 'street-hail',
                         'trip_type_2.0': 'dispatch',}, inplace= True)

In [ ]:
con_data.sample(10)

In [ ]:
con_data.info()

---
---

In [ ]:
con_data.isna().sum()

In [ ]:
# Simple Validation Method
con_data_train , con_data_test = train_test_split(con_data, test_size=0.2, random_state=199)

In [ ]:
sns.heatmap(con_data.corr())

In [ ]:
con_data.drop(columns=['ehail_fee', 'congestion_surcharge'], inplace=True)

In [ ]:
duplicate = con_data.duplicated()
print(duplicate.sum())
con_data[duplicate]

In [ ]:
con_data.drop_duplicates(inplace=True)

In [ ]:
con_data.info()

In [ ]:
#con_data=con_data[con_data['passenger_count']>=0]
#con_data.info()

In [ ]:
# fill passenger_count with mean
mean= con_data['passenger_count'].mean()

con_data['passenger_count'].fillna(mean, inplace= True)
con_data

In [ ]:
con_data.info()

In [ ]:
# trips with zero distances
con_data[con_data['trip_distance'] == 0]


In [ ]:
# removing zero distance trips
con_data = con_data[con_data['trip_distance'] > 0]
con_data.describe()

In [ ]:
con_data.shape

In [ ]:
# removing trips with zero/negative fares
con_data= con_data[con_data['fare_amount'] > 0]
con_data

In [ ]:
# removing passenger_count with zero/negative 
con_data= con_data[con_data['passenger_count'] > 0]
con_data

In [ ]:
con_data.describe()

In [ ]:
# fig1=plt.figure(figsize=(11, 8))
# ax5=fig1.add_subplot(1,1,1)
# ax5.scatter(con_data.trip_distance, con_data.fare_amount, color='green')
# ax5.set_title('The graph of payment depending on the trip_distance')
# ax5.set_xlabel('Day of the week')
# ax5.set_ylabel('Payment')


In [ ]:
# fig2=plt.figure(figsize=(11, 8))
# ax5=fig2.add_subplot(1,1,1)
# ax5.scatter(con_data.passenger_count, con_data.fare_amount, color='green')
# ax5.set_title('The graph of payment depending on the number of passengers');
# ax5.set_xlabel('Number of passengers')
# ax5.set_ylabel('Payment')

In [ ]:
# fig3=plt.figure(figsize=(11, 8))
# ax5=fig3.add_subplot(1,1,1)
# ax5.scatter(con_data.week_day_pickup, con_data.fare_amount, color='green')
# ax5.set_title('The graph of payment depending on the day of the week')
# ax5.set_xlabel('Day of the week')
# ax5.set_ylabel('Payment')